In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os


In [6]:
# Path to the folder where you saved your dataset
data_dir = r"C:\Users\Lenovo\Downloads\oxford-iiit-pet"
# Create a dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  image_size=(180, 180),
  batch_size=32)


Found 22170 files belonging to 2 classes.


In [7]:
# Get list of all file paths
all_file_paths = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir)]

# Assume that the label for each image is contained in the file name
# For example, if the file name is 'cat_001.jpg', the label is 'cat'
all_labels = [os.path.splitext(os.path.basename(fname))[0].split('_')[0] for fname in all_file_paths]

# Create a mapping of class names to integers
class_names = np.unique(all_labels)
class_to_index = {name: index for index, name in enumerate(class_names)}

In [8]:
# Now you can split this data into training and validation sets
train_file_paths = all_file_paths[:16000]
train_labels = [class_to_index[label] for label in all_labels[:16000]]

val_file_paths = all_file_paths[16000:]
val_labels = [class_to_index[label] for label in all_labels[16000:]]

In [9]:

# Define the custom data generator
def load_and_preprocess_image(path):
    try:
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [180, 180])
        image /= 255.0  # normalize to [0,1] range
    except:
        return None
    return image

def custom_data_generator(file_paths, labels, batch_size=32):
    assert len(file_paths) == len(labels)
    for i in range(0, len(file_paths), batch_size):
        batch_file_paths = file_paths[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]
        image_batch = []
        label_batch = []
        for fp, label in zip(batch_file_paths, batch_labels):
            image = load_and_preprocess_image(fp)
            if image is not None:
                image_batch.append(image)
                label_batch.append(label)
        yield np.array(image_batch), np.array(label_batch)


In [10]:
# Split the dataset into training and validation sets
train_ds = tf.data.Dataset.from_generator(
    custom_data_generator,
    args=[train_file_paths, train_labels, 32],  # replace with your actual data
    output_signature=(
        tf.TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
)

val_ds = dataset.skip(16000)


In [11]:
# Simple Model
model1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10)
])

In [12]:
# Optimized Model
model2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3), kernel_regularizer=regularizers.l2(0.001)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(10)
])


In [13]:
model1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [14]:
# Train the models
try:
    history1 = model1.fit(train_ds, validation_data=val_ds, epochs=10)
    history2 = model2.fit(train_ds, validation_data=val_ds, epochs=10)
except tf.errors.InvalidArgumentError as e:
    print(f"An error occurred during training: {e}")


Epoch 1/10


An error occurred during training: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element of shape (0,) where an element of shape (None, 180, 180, 3) was expected.
Traceback (most recent call last):

  File "C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)

  File "C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 235, in ge

In [15]:
# Now you can plot the training history
def plot_history(history, title):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(title)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

plot_history(history1, 'Model 1')
plot_history(history2, 'Model 2')

NameError: name 'history1' is not defined